# 如何使用少量示例
:::info 前提条件
本指南假定您熟悉以下概念：- [提示模板](/docs/concepts/prompt_templates)- [示例选择器](/docs/concepts/example_selectors)- [大型语言模型（LLMs）](/docs/concepts/text_llms)- [向量数据库](/docs/concepts/vectorstores)
:::
在本指南中，我们将学习如何创建一个简单的提示模板，用于在生成内容时为模型提供输入和输出的示例。向大语言模型提供少量此类示例的方法称为[小样本提示](/docs/concepts/few_shot_prompting/)，这是一种简单而有效的方式，既能引导生成过程，又能在某些情况下显著提升模型性能。
少量样本提示模板可以通过一组示例构建，也可以从[示例选择器](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.base.BaseExampleSelector.html)类构建，该类负责从定义的集合中选择示例子集。
本指南将介绍使用字符串提示模板进行少量示例学习的方法。如需了解针对聊天模型使用聊天消息进行少量示例学习的指南，请参阅[此处](/docs/how_to/few_shot_examples_chat/)。
## 为少量示例创建格式化器
配置一个格式化程序，将少量示例格式化为字符串。该格式化程序应为 `PromptTemplate` 对象。

In [1]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

## 创建示例集
接下来，我们将创建一个少样本示例列表。每个示例应为一个字典，代表我们之前定义的格式化提示模板的输入示例。

In [2]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

让我们用其中一个示例来测试格式提示：

In [13]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



### 将示例和格式化程序传递给 `FewShotPromptTemplate`
最后，创建一个 [`FewShotPromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html) 对象。该对象接收少量示例（few-shot examples）以及这些示例的格式化器。当这个 `FewShotPromptTemplate` 被格式化时，它会先使用 `example_prompt` 对传入的示例进行格式化，然后将它们添加到 `suffix` 之前的最终提示中。

In [14]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

通过为模型提供这样的示例，我们可以引导模型生成更优质的回应。

## 使用示例选择器
我们将复用上一节的示例集和格式化工具。不过，这次不会直接将示例输入到`FewShotPromptTemplate`对象中，而是将其传递给一个名为[`SemanticSimilarityExampleSelector`](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html)的`ExampleSelector`实现实例。该实现会根据输入内容与初始示例集的语义相似度来筛选少样本示例，其核心原理是通过嵌入模型计算输入内容与少样本示例之间的相似度，并借助向量数据库执行最近邻搜索。
为了展示其外观，让我们初始化一个实例并单独调用它：

In [4]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


现在，让我们创建一个 `FewShotPromptTemplate` 对象。该对象接收示例选择器以及用于少样本示例的格式化提示。

In [5]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


## 后续步骤
你现在已经学会了如何在提示中添加少量示例。
接下来，请查阅本节中关于提示模板的其他操作指南，或参考相关的[聊天模型小样本学习操作指南](/docs/how_to/few_shot_examples_chat)，以及其他[示例选择器操作指南](/docs/how_to/example_selectors/)。